In [1]:
import FaaSET
import pprint
import pandas as pd
import numpy as np
import time

pp = pprint.PrettyPrinter(indent=4)

------------------------------------------------------
Welcome to FaaSET v2.0! Some things have changed when
compared to v1.0. RunModes and the Containerize arguments
have been removed in favor of more available Platforms.
Platforms is no longer a list and now functions can
only be deployed to a single platform at a time.
If you are using a older notebook that has not been
updated things will be broken. Please fix them. 
------------------------------------------------------
Loaded platforms:  AWS. IBM Docker. AWS Docker ARM. OpenFaaS EKS. GCF Gen2. OpenFaaS. IBM. AWS Docker. AWS Docker Debian. Azure. Local. GCF. 
------------------------------------------------------
Loaded functions:  hello_world_c. cache_exp. sleep_test. computation_test. cache_exp_ibm. cache_bench_test. 


In [2]:
@FaaSET.cloud_function(platform="IBM Docker")
def cache_exp_ibm(request, context): 
    return {"message": "Dummy Function"}   

# cache_exp_ibm({}, None)

In [3]:
cache_exp_ibm({}, None)

{'message': 'Hello world'}

In [29]:
%%writefile functions/cache_exp_ibm/Dockerfile

#FROM python:buster as build-image
FROM ibmfunctions/action-python-v3.7

# Update and install curl. 
RUN apt update
RUN apt install -y curl build-essential moreutils expect# stdbuf

# Define and create function directory
ENV FUNCTION_DIR="/var/task"
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY . ${FUNCTION_DIR}

# RUN pip install boto3 botocore

# # Install the runtime interface client
# RUN pip install \
#         --target ${FUNCTION_DIR} \
#         awslambdaric



# Set working directory
WORKDIR ${FUNCTION_DIR}

# Compile C code
# RUN gcc -o stream.10M -O -DSTREAM_ARRAY_SIZE=10000000 -DNTIMES=10 stream_small.c
RUN cd ${FUNCTION_DIR}/llcbench  && make clean && make cache-bench 
# RUN gcc cpu_bound.c -o cpu_bound


# Set entry script that determines whether to run RIE if running locally
# COPY ./entry_script.sh /entry_script.sh
# RUN chmod +x /entry_script.sh
# ENTRYPOINT [ "/entry_script.sh" ]

# Set handler
# CMD [ "lambda_function.lambda_handler" ]

Overwriting functions/cache_exp_ibm/Dockerfile


In [48]:
@FaaSET.cloud_function(platform="IBM Docker")
def cache_exp_ibm(request, context): 
    import subprocess, datetime, time

    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
    
#     command = "stdbuf -i0 -o0 -e0 /var/task/llcbench/cachebench/cachebench -w -d5 -x1 -m31 -e1 | ts '[%Y-%m-%d %H:%M:%.S]'"
    command = "/var/task/llcbench/cachebench/cachebench -w -d5 -x1 -m30 -e1"
#     start_time = float(request['start_time'])
#     print("start time is: {}".format(start_time))
    
#     now = datetime.datetime.now().timestamp()
#     if start_time >= now:
#         time.sleep(start_time - now)
    output = subprocess.check_output(command.split()).decode('ascii')

    
    inspector.addAttribute("cache_bench_metric", output)
    inspector.inspectAllDeltas()
    return inspector.finish()

In [50]:
%time ret = cache_exp_ibm({}, None)

An exception occurred reading the response:
--->Expecting value: line 2 column 1 (char 1)
---> Command: ['./functions/cache_exp_ibm/run.sh', './functions/cache_exp_ibm/', '{}']
---> Response: 

---> Error: ok: invoked /_/cache_exp_ibm, but the request has not yet finished, with id 05386eb359d24a29b86eb359d20a291a

---> Stack Trace:
CPU times: user 6.19 ms, sys: 17.9 ms, total: 24.1 ms
Wall time: 1min 1s


Traceback (most recent call last):
  File "/home/itadmin/c_example/SAAF/jupyter_workspace/FaaSET.py", line 107, in test
    obj = json.loads(out)
  File "/home/itadmin/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/itadmin/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/itadmin/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 2 column 1 (char 1)


In [13]:
import datetime
now = datetime.datetime.now().timestamp()

In [14]:
%time ret = computation_test({'start_time': int(now + 3)}, None)

CPU times: user 15.5 ms, sys: 15.2 ms, total: 30.6 ms
Wall time: 4min 2s


In [15]:
ret

{'version': 0.6,
 'lang': 'python',
 'startTime': 1658450441663,
 'invocations': 1,
 'initializationTime': 1658450441660,
 'uuid': '0d48874c-b36f-4160-8818-ebb187e9596f',
 'newcontainer': 1,
 'cpuType': 'Intel(R) Xeon(R) Processor @ 2.50GHz',
 'cpuModel': '63',
 'architecture': 'x86',
 'cpuCores': 2,
 'cpuInfo': [{'processor': '0',
   'vendor_id': 'GenuineIntel',
   'cpu_family': '6',
   'model': '63',
   'model_name': 'Intel(R) Xeon(R) Processor @ 2.50GHz',
   'stepping': '2',
   'microcode': '0x1',
   'cpu_MHz': '2500.010',
   'cache_size': '33792 KB',
   'physical_id': '0',
   'siblings': '2',
   'core_id': '0',
   'cpu_cores': '2',
   'apicid': '0',
   'initial_apicid': '0',
   'fpu': 'yes',
   'fpu_exception': 'yes',
   'cpuid_level': '13',
   'wp': 'yes',
   'flags': ['fpu',
    'vme',
    'de',
    'pse',
    'tsc',
    'msr',
    'pae',
    'mce',
    'cx8',
    'apic',
    'sep',
    'mtrr',
    'pge',
    'mca',
    'cmov',
    'pat',
    'pse36',
    'clflush',
    'mmx',
  

In [18]:
ret['runtime'] / 60000

3.9907666666666666

In [19]:
ret['cpuStealDelta']

200.0

In [20]:
ret['cpuStealDelta'] / 4

50.0

In [22]:
@FaaSET.cloud_function(platform="AWS Docker Debian")
def sleep_test(request, context): 
    return {"message": "Dummy Function"}   

sleep_test({}, None)

Build: Loading config...
Build: Purging previous build...
Build: Copying files...
Build: Cleaning up...
Build: Building Docker Image...
Sending build context to Docker daemon  23.55kB

Step 1/15 : FROM python:3.8-slim-buster
 ---> c706fbd78639
Step 2/15 : RUN apt update
 ---> Using cache
 ---> 9e15b7761466
Step 3/15 : RUN apt install -y curl
 ---> Using cache
 ---> 3d20f521c30d
Step 4/15 : ENV FUNCTION_DIR="/var/task"
 ---> Using cache
 ---> e610f09d2e1d
Step 5/15 : RUN mkdir -p ${FUNCTION_DIR}
 ---> Using cache
 ---> 7e959d5543b4
Step 6/15 : COPY . ${FUNCTION_DIR}
 ---> Using cache
 ---> 52065eaeb552
Step 7/15 : RUN pip install boto3 botocore
 ---> Using cache
 ---> 9b611accbdcf
Step 8/15 : RUN pip install         --target ${FUNCTION_DIR}         awslambdaric
 ---> Using cache
 ---> 47774a69503f
Step 9/15 : RUN curl -Lo /usr/local/bin/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie
 ---> Using cache
 ---> b3d53d82a195

Traceback (most recent call last):
  File "/home/itadmin/c_example/SAAF/jupyter_workspace/FaaSET.py", line 107, in test
    obj = json.loads(out)
  File "/home/itadmin/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/home/itadmin/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/home/itadmin/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)


{}

In [29]:
%%writefile functions/sleep_test/Dockerfile

#FROM python:buster as build-image
FROM python:3.8-slim-buster

# Update and install curl. 
RUN apt update
RUN apt install -y curl build-essential moreutils expect

# Define and create function directory
ENV FUNCTION_DIR="/var/task"
RUN mkdir -p ${FUNCTION_DIR}

# Copy function code
COPY . ${FUNCTION_DIR}

RUN pip install boto3 botocore

# Install the runtime interface client
RUN pip install \
        --target ${FUNCTION_DIR} \
        awslambdaric

# Install the runtime interface emulator
RUN curl -Lo /usr/local/bin/aws-lambda-rie https://github.com/aws/aws-lambda-runtime-interface-emulator/releases/latest/download/aws-lambda-rie
RUN chmod +x /usr/local/bin/aws-lambda-rie

# Set working directory
WORKDIR ${FUNCTION_DIR}

# Compile C code
# RUN gcc -o stream.10M -O -DSTREAM_ARRAY_SIZE=10000000 -DNTIMES=10 stream_small.c
# RUN cd ${FUNCTION_DIR}/llcbench  && make clean && make cache-bench 
# RUN gcc cpu_bound.c -o cpu_bound


# Set entry script that determines whether to run RIE if running locally
COPY ./entry_script.sh /entry_script.sh
RUN chmod +x /entry_script.sh
ENTRYPOINT [ "/entry_script.sh" ]

# Set handler
CMD [ "lambda_function.lambda_handler" ]

Overwriting functions/sleep_test/Dockerfile


In [30]:
@FaaSET.cloud_function(platform="AWS Docker Debian", config={"memory": 259})
def sleep_test(request, context): 
    import subprocess, datetime, time

    from SAAF import Inspector
    inspector = Inspector()
    inspector.inspectAll()
    
#     command = "/var/task/llcbench/cachebench/cachebench -w -d1 -x1 -m20 -e1"

    command = "sleep 4m" 
#     command = "stdbuf -i0 -o0 -e0 /var/task/llcbench/cachebench/cachebench -w -d1 -x1 -m10 -e1 | ts '[%Y-%m-%d %H:%M:%.S]'"
    start_time = float(request['start_time'])
    print("start time is: {}".format(start_time))
    
    now = datetime.datetime.now().timestamp()
    if start_time >= now:
        time.sleep(start_time - now)
        
    output = subprocess.check_output(command.split()).decode('ascii')

#     inspector.addAttribute("cache_bench_metric", output)
    inspector.inspectAllDeltas()
    return inspector.finish()

In [33]:
now = datetime.datetime.now().timestamp()
%time ret2 = sleep_test({'start_time': int(now + 3)}, None)

CPU times: user 19.4 ms, sys: 7.83 ms, total: 27.2 ms
Wall time: 4min 2s


In [34]:
ret2

{'version': 0.6,
 'lang': 'python',
 'startTime': 1658451556163,
 'invocations': 1,
 'initializationTime': 1658451556158,
 'uuid': '6a449c55-ebab-48b8-a543-38782fc49571',
 'newcontainer': 1,
 'cpuType': 'Intel(R) Xeon(R) Processor @ 2.50GHz',
 'cpuModel': '63',
 'architecture': 'x86',
 'cpuCores': 2,
 'cpuInfo': [{'processor': '0',
   'vendor_id': 'GenuineIntel',
   'cpu_family': '6',
   'model': '63',
   'model_name': 'Intel(R) Xeon(R) Processor @ 2.50GHz',
   'stepping': '2',
   'microcode': '0x1',
   'cpu_MHz': '2500.012',
   'cache_size': '36608 KB',
   'physical_id': '0',
   'siblings': '2',
   'core_id': '0',
   'cpu_cores': '2',
   'apicid': '0',
   'initial_apicid': '0',
   'fpu': 'yes',
   'fpu_exception': 'yes',
   'cpuid_level': '13',
   'wp': 'yes',
   'flags': ['fpu',
    'vme',
    'de',
    'pse',
    'tsc',
    'msr',
    'pae',
    'mce',
    'cx8',
    'apic',
    'sep',
    'mtrr',
    'pge',
    'mca',
    'cmov',
    'pat',
    'pse36',
    'clflush',
    'mmx',
  

In [35]:
ret2['cpuStealDelta']

10.0